In [4]:
import time
import math
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import zipfile

import sys
sys.path.append("..") 
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def load_data_jay_lyrics():
    """加载周杰伦歌词数据集"""
    with zipfile.ZipFile('data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

# 定义模型

构造一个含单隐藏层、隐藏单元个数为256的循环神经网络层rnn_layer

其输入形状为：**(时间步数, 批量大小, 输入个数)**。

其中，输入个数即one-hot向量长度（词典大小），

rnn_layer在前向计算后回分别返回输出和隐藏状态h，其中输出指的是隐藏层在各个时间步上计算并输出的隐藏状态，它们通常作为后续输出层的输入。

输出形状为**(时间步数, 批量大小, 隐藏单元个数)**。

隐藏状态h的形状为**(层数, 批量大小, 隐藏单元个数)**。

## 定义循环神经网络层rnn_layer

In [20]:
num_hiddens = 256
rnn_layer = nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)

## 定义完整模型

In [24]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.vocab_size = vocab_size
        self.dense = nn.Linear(self.hidden_size, vocab_size)
        self.state = None
    
    def forward(self, inputs, state): # inputs: (batch, seq_len)
        # 获取one-hot向量表示
        X = d2l.to_onehot(inputs,self.vocab_size) #X是个List
        Y, self.state = self.rnn(torch.stack(X), state)
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，
        # 它的输出形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.view(-1, Y.shape[-1]))
        return output, self.state
        

# 预测函数
在训练之前，先定义一个预测函数，与6-4节的区别在于前向计算和初始化隐藏状态的函数接口。

In [26]:
def predict_rnn_pytorch(prefix, num_chars, model, vocab_size,device,
                       idx_to_char,char_to_idx):
    state = None # 初始时的隐藏状态
    output = [char_to_idx[prefix[0]]] # output会记录prefix加上输出
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1,1)
        if state is not None:
            if isinstance(state,tuple): # LSTM,state:(h,c)
                state = (state[0].to(device), state[1].to(device))
            else:
                state = state.to(device)
        
        (Y,state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [27]:
model = RNNModel(rnn_layer, vocab_size).to(device)
predict_rnn_pytorch('分开', 10, model, vocab_size, device, idx_to_char, char_to_idx)

'分开姑远娇育娇育娇育娇育'

# 训练函数

In [28]:
def train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state=None
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(corpus_indices, batch_size, num_steps, device) # 相邻采样
        for X, Y in data_iter:
            if state is not None:
                # 使用detach函数从计算图分离隐藏状态, 这是为了
                # 使模型参数的梯度计算只依赖一次迭代读取的小批量序列(防止梯度计算开销太大)
                if isinstance (state, tuple): # LSTM, state:(h, c)  
                    state = (state[0].detach(), state[1].detach())
                else:   
                    state = state.detach()

            (output, state) = model(X, state) # output: 形状为(num_steps * batch_size, vocab_size)

            # Y的形状是(batch_size, num_steps)，转置后再变成长度为
            # batch * num_steps 的向量，这样跟输出的行一一对应
            y = torch.transpose(Y, 0, 1).contiguous().view(-1)
            l = loss(output, y.long())

            optimizer.zero_grad()
            l.backward()
            # 梯度裁剪
            d2l.grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        try:
            perplexity = math.exp(l_sum / n)
        except OverflowError:
            perplexity = float('inf')
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, perplexity, time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_pytorch(
                    prefix, pred_len, model, vocab_size, device, idx_to_char,
                    char_to_idx))

In [29]:
num_epochs, batch_size, lr, clipping_theta = 250, 32, 1e-3, 1e-2 # 注意这里的学习率设置
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']
train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)


epoch 50, perplexity 11.667677, time 0.49 sec
 - 分开始着我 别怪我 说你的手 一直的口 你有一个 你的手 我不的这样打我妈  不要再想 我不要再想 我想
 - 不分开 我想要你已经离开  不知不觉 我想你这样打我妈  不要再想 我不要再想 我想要你想你不你 想要你你
epoch 100, perplexity 1.270971, time 0.50 sec
 - 分开不了口 周杰伦 语 走 娘 我有多不着 让我心愿的看远方  不能听永远止当的手牵着你的手 一阵莫名感
 - 不分开这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难道你手不会痛吗 我叫你这样打我妈妈 难道你手不
epoch 150, perplexity 1.063952, time 0.49 sec
 - 分开 我去 爸爱你看的从简 我不能看着你的手 一阵落 吹完 我出你 漂亮的让我面红的可爱女人 温柔的让我
 - 不分开这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难道你手不会痛吗 我叫你爸棒你打我妈 你知道这 
epoch 200, perplexity 1.033415, time 0.49 sec
 - 分开 我不 爸爱你看我球简单我这样打我妈妈  道知不 不会听 不想不想不你不这 你跟了它可以前 让温能么
 - 不分开这样打我妈妈 难道你手不会痛吗 不要再这样打我妈妈 难道你手不会痛吗 我叫你爸 你打我妈 这样对吗干
epoch 250, perplexity 1.020715, time 0.51 sec
 - 分开 我不 爸 你打我妈妈妈  爱分球 别痛 我说下了 你是我承受我 泪不休 语沉默 娘子却依旧每日折一
 - 不分开  像知不觉 我跟了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 
